# An Analysis of the Berlin Marathon Runners
- What is the Berlin marathon?
    - The Berlin marathon is a global marathon held yearly in Berlin, Germany since 1974. The dataset in used includes running times since the marathon's inception, and we delve into how some trends in running speeds have changed over time, and what factors play into lower running times.
- How have the average/fastest times changed over the years?
    - The top left graph shows that the average running times have increased over the years, likely due to an increase in the interest in the running community. The final running times, however, have decrease, given the increased number of challengers.
- How does age and gender affect running times
    - In the bottom left, we can see that women are overall slower than men by 30 minutes on average. In terms of age (the graph can be viewed in the dropdown), the peak running times occur around 30 years.
- Where are most runners from? Where are the fastest from. 
    - Given that this event takes place in Berlin, most runners are from Germany. The fastest runners are from Kenya, Ethiopia. As studies have now shown, this is due to strong genetics and high-altitude training.

This project takes the Berlin Marathon dataset and analyzes it look at how running times have changed over the years, and over different age groups and genders. It also takes a look into where the top runners are from.

In [11]:
import pandas as pd
import matplotlib.pyplot as plt
import panel as pn
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

pn.extension('tabulator')

import hvplot.pandas


In [12]:
#format: year, country, gender, age, time
#(all participants)
berlin_marathon = pd.read_csv('1_running_datasets/berlin_marathon_1974_2023.csv')

age_null_values = ['H', 'JU20', 'L1', 'L2', 'L', 'L3', 'D1', 'L4', 'D3', 'D2',
                  'BM','A', 'B', 'C', 'DH', 'DA', 'DB', 'DJ', 'M0', 'Ber', 'M',
                  'M<', 'Jug']

#data cleaning
berlin_marathon = berlin_marathon[~(berlin_marathon['time'].isin(['–', 'DSQ', 'no time']))&
                                  ~(berlin_marathon['age'].isin(age_null_values))]
#convert time to hrs
berlin_marathon['time_hrs'] = pd.to_timedelta(berlin_marathon['time']).dt.total_seconds() / 3600
berlin_marathon.dropna(subset = 'time_hrs', how='any')
berlin_marathon['age'] = pd.to_numeric(berlin_marathon['age'])

/var/folders/_p/y56kr6vs1fz7ycsbxhyqwwwc0000gn/T/ipykernel_93573/829007302.py:3: DtypeWarning: Columns (1,3) have mixed types. Specify dtype option on import or set low_memory=False.
  berlin_marathon = pd.read_csv('1_running_datasets/berlin_marathon_1974_2023.csv')


In [13]:
year_slider = pn.widgets.IntSlider(name='Year slider', start=1974, end=2023, step=1, value=2023)
idf = berlin_marathon.interactive()

## Average and Finishing times
- How have average times changed over the years?
    - Overall average time has increased. Likely due to the increase in interest in the sport.
- How has the fastest time changed over the years?
    - Finishing times have decreased. obvs. because records are meant to be broken.

In [14]:
overall_times = pd.DataFrame(berlin_marathon.groupby('year')['time_hrs'].mean().rename('Average Times'))#.hvplot(kind = 'scatter', title = 'Average times over the years')
overall_times['Finishing times'] = berlin_marathon.groupby('year')['time_hrs'].min()
overall_times_plot = overall_times.hvplot(title = 'Average and Finishing times', xlabel = 'year', ylabel = 'time(hrs)')
overall_times_plot

:NdOverlay   [Variable]
   :Curve   [year]   (value)

## Table df

In [15]:
# berlin_marathon.interactive()

In [27]:
marathon_table = berlin_marathon.sort_values(by = 'time_hrs').drop(columns = ['time_hrs']).dropna(subset = 'country').interactive().reset_index()
marathon_table = marathon_table.pipe(pn.widgets.Tabulator, page_size = 10, sizing_mode='stretch_width')
marathon_table

## How does gender affect running times?

In [17]:
berlin_marathon.dtypes

year          int64
country      object
gender       object
age         float64
time         object
time_hrs    float64
dtype: object

In [18]:

age_data_pipeline = (
    berlin_marathon
    .sort_values(by = 'year', ascending = False)
    .groupby(['age'])['time_hrs'].mean()
    .to_frame()
    .reset_index()
    .reset_index(drop=True)

)
age_data_plot = age_data_pipeline.hvplot(
    kind = 'scatter',
    title = 'Average time by age', 
    x = 'age',
    y = 'time_hrs')
age_data_plot

:Scatter   [age]   (time_hrs)

## How does gender affect times?

In [19]:
gender_data = berlin_marathon[(berlin_marathon['gender'] != 'X') & (berlin_marathon['gender'] != '–') ]
gender_data_pipeline = (
    gender_data
    .sort_values(by = 'year')
    .groupby(['gender', 'year'])['time_hrs'].mean()
    .to_frame()
    .reset_index()
    .reset_index(drop=True)

)
gender_data_plot = gender_data_pipeline.hvplot(
    kind = 'scatter',
    by = 'gender',
    title = 'Average time by gender', 
    x = 'year',
    y = 'time_hrs')
gender_data_plot

:NdOverlay   [gender]
   :Scatter   [year]   (time_hrs)

### What are the average time differences

In [20]:
time_diff = idf[(idf['gender'] != 'X') & (idf['gender'] != '–') ]
time_diff = time_diff.groupby('gender')['time_hrs'].mean().reset_index()
time_diff.iloc[0,1]-time_diff.iloc[1,1]


## Where are most runners from?
- Where do most runners come from?
    - Germany
- Where are the shortest times from?
    - Kenya, followed by Ethiopia. Even tho Germany had the most participants ahaha
- Are these two places the same?
    - no. no they are not.

In [21]:
runners_count = berlin_marathon.groupby(['country', 'gender']).size().sort_values(ascending = False)
runners_count = runners_count[runners_count > 1000]
runners_count_plot = runners_count.hvplot(kind = 'bar', 
                                               stacked = True, 
                                               rot = 45,
                                               title = 'Where are most runners from?')
runners_count_plot

:Bars   [country,gender]   (0)

## Where are the top 50 runners from?

In [22]:
gender_selector = pn.widgets.Select(options=['All', 'male', 'female'], value='Men', name='Select Gender')
gender_selector

Select(options=['All', 'male', 'female'], value='All')

In [23]:
gender_selector = pn.widgets.Select(options=['male', 'female'], value='Men', name='Select Gender')
top_50_pipeline = (
    idf[
        (idf['gender'] == gender_selector)
    ]
    .dropna(subset=['country', 'gender']).sort_values(by = 'time_hrs')[:50]
    .groupby(['country', 'gender']).size()
    .sort_values(ascending = False)
    .to_frame()
    .reset_index()
    .reset_index(drop=True)
)
top_50_plot = top_50_pipeline.hvplot(kind = 'bar', 
                       x = 'country',
                       ylabel = 'count',
                       stacked = True, 
                       rot = 45,
                       title = 'Where are the top 50 runners from?')
top_50_plot

In [24]:
plot_selector_2_1 = pn.widgets.Select(options=['Average time by gender', 'Average time by age'], name='Select Plot')
@pn.depends(plot_selector_2_1.param.value)
def reactive_plot_2_1(selected_plot):
    
    if selected_plot == 'Average time by gender':
        return gender_data_plot
    elif selected_plot == 'Average time by age':
        return age_data_plot

plot_selector_2_2 = pn.widgets.Select(options=['Top 50', 'Runner Population'], name='Select Plot')
@pn.depends(plot_selector_2_2.param.value)
def reactive_plot_2_2(selected_plot):
    if selected_plot == 'Top 50':
        return top_50_plot
    elif selected_plot == 'Runner Population':
        return runners_count_plot


In [54]:
template = pn.template.FastListTemplate(
    title = 'Berlin Marathon runners',
    sidebar = [pn.pane.Markdown("# Berlin Marathon!"),
               pn.pane.Markdown("### What is the Berlin marathon? \n "
                                " - The Berlin marathon is a global marathon held yearly in Berlin, Germany since 1974."
                                "The dataset in used includes running times since the marathon's inception, "
                                "and we delve into how some trends in running speeds have changed over time, "
                                "and what factors play into lower running times."),
               pn.pane.Markdown("### How have the average and fastest times changed over the years? \n"
                               " - The top left shows a comparison of the average and the fastest times, which have increased"
                               "over the years. This is likely due to an increase in the interest in the running community. "
                               "The final running times, however, have decreased, given the increased number of challengers. "),
               pn.pane.Markdown("### How does age and gender affect running times? \n"
                               " - In the bottom left, we can see that women are overall slower than men by 30 minutes on "
                               "average. In terms of age (the graph can be viewed in the dropdown), the peak running times "
                               "occur around 30 years."),
               pn.pane.Markdown("### Where are most runners from? \n"
                               " - Given that this event takes place in Berlin, most runners are from Germany. The fastest"
                               "runners are from Kenya and Ethiopia. As studies have now shown, this is due to strong genetics and "
                               "high-altitude training.")
              ],
    main = [pn.Row(pn.Column(overall_times_plot), pn.Column(marathon_table)),#, top_50_plot)),
           pn.Row(pn.Column(plot_selector_2_1, reactive_plot_2_1), pn.Column(plot_selector_2_2, reactive_plot_2_2))
           ])
template.servable()


FastListTemplate
    [js_area] HTML(None, height=0, margin=0, sizing_mode='fixed', width=0)
    [actions] TemplateActions()
    [browser_info] BrowserInfo(dark_mode=True, device_pixel_ratio=1.7999999523162842, language='en-CA', timezone='America/Toronto', timezone_offset=300, webdriver=False)
    [busy_indicator] LoadingSpinner(height=20, width=20)
    [10931884176] Row
        [0] Column
            [0] HoloViews(NdOverlay, height=300, sizing_mode='fixed', width=700)
        [1] Column
            [0] Interactive(Interactive)
    [10932875216] Row
        [0] Column
            [0] Select(options=['Average time by gender',...], value='Average time by gender')
            [1] ParamFunction(function, _pane=HoloViews, defer_load=False)
        [1] Column
            [0] Select(options=['Top 50', 'Runner P...], value='Top 50')
            [1] ParamFunction(function, _pane=Interactive, defer_load=False)
    [10931995600] Markdown(str)
    [10931772432] Markdown(str)
    [10932724560] Markdown(str)
    [10932790736] Markdown(str)
    [10932654800] Markdown(str)